In [47]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
import time

In [74]:
# transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=128, shuffle=False)

In [75]:
class Encoder(nn.Module):
    def __init__(self, message_size):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, message_size)

    def forward(self, picture):
        picture = torch.flatten(picture, start_dim=1)
        picture = torch.relu(self.fc1(picture))
        return self.fc2(picture)

In [76]:
class Decoder(nn.Module):
    def __init__(self, message_size):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(message_size, 512)
        self.fc2 = nn.Linear(512, 28 * 28)

    def forward(self, picture):
        picture = torch.relu(self.fc1(picture))
        picture = torch.sigmoid(self.fc2(picture))
        return picture.reshape((-1, 1, 28, 28))

In [77]:
class Autoencoder(nn.Module):
    def __init__(self, message_size):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(message_size)
        self.decoder = Decoder(message_size)

    def forward(self, picture):
        picture = self.encoder(picture)
        return self.decoder(picture)

In [155]:
def train(autoencoder_model, epochs=5, step=0.1):
    criterion = nn.MSELoss() # nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(autoencoder_model.parameters(), lr=step)
    for epoch in range(epochs):
        running_loss = 0.0
        for i, (images, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = autoencoder_model(images)
            # loss = ((images - outputs) ** 2).sum()
            loss = criterion(outputs, images)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * images.size(0)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}")

In [156]:
autoencoder = Autoencoder(28)
train(autoencoder, epochs=10, step=0.001)
# torch.save(autoencoder.state_dict(), "autoencoder.pth")

Epoch [1/10], Loss: 4.22429252922662
Epoch [2/10], Loss: 1.4875383229652193
Epoch [3/10], Loss: 1.1726971417983203
Epoch [4/10], Loss: 1.0406302268317005
Epoch [5/10], Loss: 0.9571864700266547
Epoch [6/10], Loss: 0.897565286900443
Epoch [7/10], Loss: 0.8510420710039037
Epoch [8/10], Loss: 0.8150464718593463
Epoch [9/10], Loss: 0.7870395601685367
Epoch [10/10], Loss: 0.7615064760324544


In [157]:
def assessment(autoencoder_model):
    total_loss = 0.0
    num_samples = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    autoencoder_model.eval()
    with torch.no_grad():
        for images, _ in test_loader:
            images = images.to(device)
            reconstructed_images = autoencoder_model(images)
            loss = F.mse_loss(reconstructed_images, images, reduction='sum')
            total_loss += loss.item()
            num_samples += images.size(0)
    mean_loss = total_loss / num_samples

    return mean_loss

In [158]:
print(f'Потери автоенкодера при проверке на тестовом наборе: {assessment(autoencoder):.2f}%')

Потери автоенкодера при проверке на тестовом наборе: 4.71%


In [159]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten() # ф-ция преобразование изображения в вектор
        self.fc1 = nn.Linear(28 * 28, 512) # полносвязанный слой с 128 нейронами
        self.fc2 = nn.Linear(512, 10) # выходной слой с 10 нейронами (для 10 цифр)

    def forward(self, x):
        x = self.flatten(x) # разворачиваем в один вектор
        x = torch.relu(self.fc1(x)) # Применение ReLU к выходу первого слоя
        return self.fc2(x) # Применение второго полносвязанного слоя

In [160]:
digit_recognizer = SimpleNN()
#autoencoder.load_state_dict(torch.load("autoencoder.pth"))
with torch.no_grad():
    digit_recognizer.fc1.weight = autoencoder.encoder.fc1.weight
    digit_recognizer.fc1.bias = autoencoder.encoder.fc1.bias
    digit_recognizer.fc2.weight = autoencoder.encoder.fc2.weight
    digit_recognizer.fc2.bias = autoencoder.encoder.fc2.bias

In [161]:
def assessment2(model):
    model.eval()
    as_total, as_correct = 0, 0
    with torch.no_grad():
        for as_images, as_labels in test_loader:
            as_outputs = model(as_images)
            _, as_predicted = torch.max(as_outputs.data, 1)
            as_total += as_labels.size(0)
            as_correct += (as_predicted == as_labels).sum().item()

        as_accuracy = 100 * as_correct / as_total
    return as_accuracy

In [162]:
print(f'Точность FC модели при проверке на тестовом наборе: {assessment2(digit_recognizer):.2f}%')

Точность FC модели при проверке на тестовом наборе: 1.37%


In [163]:
def fc_learn(fc_model, epochs=5, step=0.1):
    fc_criterion = nn.CrossEntropyLoss()
    fc_optimizer = torch.optim.Adam(fc_model.parameters(), lr=step)
    for epoch in range(epochs):
        fc_model.train()
        for fc_i, (fc_images, fc_labels) in enumerate(train_loader):
            fc_optimizer.zero_grad()  # Обнуление градиентов
            fc_outputs = fc_model(fc_images)  # Получение выхода модели
            fc_loss = fc_criterion(fc_outputs, fc_labels)  # Вычисление потерь
            fc_loss.backward()  # Обратное распространение ошибки
            fc_optimizer.step()  # Обновление весов

In [164]:
fc_learn(digit_recognizer, 2, 0.01)
print(f'Точность FC модели при проверке на тестовом наборе: {assessment2(digit_recognizer):.2f}%')

Точность FC модели при проверке на тестовом наборе: 96.84%
